In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=61c01ebf377b4888b6f207c83a8b4ba30b049557e968c849eae7cc04f6d70a8b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
!pip install sparkxgb

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 22.7 MB/s eta 0:00:00
  Created wheel for sparkxgb: filename=sparkxgb-0.1-py3-none-any.whl size=5629 sha256=4dbe328c2466e0a889c42b89442f4bc20280ad312b53d816597d2a3f78f624ef
  Stored in directory: /root/.cache/pip/wheels/b7/0c/a1/786408e13056fabeb8a72134e101b1e142fc95905c7b0e2a71
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767583 sha256=b368c04242fa4216b5aeddcd439709ef9b8f9a72d647d4609b44fc82a3ddb2e9
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built sparkxgb pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Fou

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, isnan
from pyspark.ml.feature import VectorAssembler, PolynomialExpansion
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Imputer

# Initialize Spark session
spark = SparkSession.builder.appName("FlightDelayPred").getOrCreate()

# Load the data
data_path = "/content/drive/MyDrive/Datasets/1987.csv"  # Replace with your file path
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Data type casting and handling missing values
df = df.withColumn("DepTime", col("DepTime").cast(FloatType()))
df = df.withColumn("Distance", col("Distance").cast(FloatType()))
df = df.withColumn("CRSDepTime", col("CRSDepTime").cast(FloatType()))
df = df.withColumn("Month", col("Month").cast(FloatType()))
df = df.withColumn("ArrDelay", col("ArrDelay").cast(FloatType()))
df = df.withColumn("DepDelay", col("DepDelay").cast(FloatType()))
df = df.withColumn("CRSElapsedTime", col("CRSElapsedTime").cast(FloatType()))
df = df.withColumn("DayofMonth", col("DayofMonth").cast(FloatType()))
df = df.withColumn("FlightNum", col("FlightNum").cast(FloatType()))

# Remove rows with null or NaN values in target column
df = df.filter(df.ArrDelay.isNotNull() & (~isnan(df.ArrDelay)))

# Selecting the features and target variable
features = ['Month', 'CRSDepTime', 'DepTime', 'Distance', 'DepDelay', 'CRSElapsedTime','DayofMonth','FlightNum']
target = 'ArrDelay'

imputer = Imputer(
    inputCols = features,
    strategy = 'median'
)

# VectorAssembler to combine feature columns into a single vector column
assembler = VectorAssembler(inputCols=features, outputCol="features", handleInvalid="skip")

# Polynomial Expansion for degree 5
polyExpansion = PolynomialExpansion(degree=5, inputCol="features", outputCol="polyFeatures")

# Define Linear Regression model for polynomial regression
poly_lr = LinearRegression(featuresCol="polyFeatures", labelCol=target)

# Update pipeline for Polynomial Regression
poly_pipeline = Pipeline(stages=[assembler, polyExpansion, poly_lr])

# Define the models
rf = RandomForestRegressor(featuresCol="features", labelCol=target)
lr = LinearRegression(featuresCol="features", labelCol=target)
gbt = GBTRegressor(featuresCol="features", labelCol=target)

# Pipelines for the models
rf_pipeline = Pipeline(stages=[assembler, rf])
lr_pipeline = Pipeline(stages=[assembler, lr])
gbt_pipeline = Pipeline(stages=[assembler, gbt])

#models = [rf_pipeline, lr_pipeline, gbt_pipeline, poly_pipeline]
models = [rf_pipeline]

for model in models:
    # Train the model
    trained_model = model.fit(df)

    # Make predictions
    predictions = trained_model.transform(df)

    # Evaluate the model for RMSE
    rmse_evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = rmse_evaluator.evaluate(predictions)

    # Evaluate the model for R2
    r2_evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    r2 = r2_evaluator.evaluate(predictions)

    # Print the RMSE and R2
    if isinstance(model.getStages()[-1], LinearRegression) and len(model.getStages()) > 2:
        model_name = "Polynomial Linear Regression"
    else:
        model_name = model.getStages()[-1].__class__.__name__
    print(f"{model_name} - Root Mean Squared Error (RMSE): {rmse}, R2: {r2}")

# Stop the Spark session
spark.stop()

RandomForestRegressor - Root Mean Squared Error (RMSE): 16.817192711743353, R2: 0.5754737491427673


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, isnan
from pyspark.ml.feature import VectorAssembler, PolynomialExpansion
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Imputer

# Initialize Spark session
spark = SparkSession.builder.appName("FlightDelayPred").getOrCreate()

# Load the data
data_path = "/content/drive/MyDrive/SENG550/2008.csv"  # Replace with your file path
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Data type casting and handling missing values
df = df.withColumn("DepTime", col("DepTime").cast(FloatType()))
df = df.withColumn("Distance", col("Distance").cast(FloatType()))
df = df.withColumn("CRSDepTime", col("CRSDepTime").cast(FloatType()))
df = df.withColumn("Month", col("Month").cast(FloatType()))
df = df.withColumn("ArrDelay", col("ArrDelay").cast(FloatType()))
df = df.withColumn("DepDelay", col("DepDelay").cast(FloatType()))
df = df.withColumn("CRSElapsedTime", col("CRSElapsedTime").cast(FloatType()))
df = df.withColumn("DayofMonth", col("DayofMonth").cast(FloatType()))
df = df.withColumn("FlightNum", col("FlightNum").cast(FloatType()))

# Remove rows with null or NaN values in target column
df = df.filter(df.ArrDelay.isNotNull() & (~isnan(df.ArrDelay)))

# Split the data into training and test sets (e.g., 80% training and 20% testing)
train_df, test_df = df.randomSplit([0.75, 0.25], seed=42)

# Selecting the features and target variable
features = ['Month', 'CRSDepTime', 'DepTime', 'Distance', 'DepDelay', 'CRSElapsedTime', 'DayofMonth', 'FlightNum']
target = 'ArrDelay'

imputer = Imputer(
    inputCols = features,
    strategy = 'mean'
)

# VectorAssembler to combine feature columns into a single vector column
assembler = VectorAssembler(inputCols=features, outputCol="features", handleInvalid="skip")


# Polynomial Expansion for degree 5
polyExpansion = PolynomialExpansion(degree=5, inputCol="features", outputCol="polyFeatures")
# Define Linear Regression model for polynomial regression
poly_lr = LinearRegression(featuresCol="polyFeatures", labelCol=target)
# Update pipeline for Polynomial Regression
poly_pipeline = Pipeline(stages=[assembler, polyExpansion, poly_lr])


# Define the models
rf = RandomForestRegressor(featuresCol="features", labelCol=target)
lr = LinearRegression(featuresCol="features", labelCol=target)
gbt = GBTRegressor(featuresCol="features", labelCol=target)

# Pipelines for the models
rf_pipeline = Pipeline(stages=[assembler, rf])
lr_pipeline = Pipeline(stages=[assembler, lr])
gbt_pipeline = Pipeline(stages=[assembler, gbt])

# List of models to train and evaluate
models = [rf_pipeline,lr_pipeline,gbt_pipeline,poly_pipeline]

for model in models:
    # Train the model on the training set
    trained_model = model.fit(train_df)

    # Make predictions on the test set
    predictions = trained_model.transform(test_df)

    # Evaluate the model for RMSE
    rmse_evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = rmse_evaluator.evaluate(predictions)

    # Evaluate the model for R2
    r2_evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    r2 = r2_evaluator.evaluate(predictions)

    # Print the RMSE and R2
    model_name = model.getStages()[-1].__class__.__name__
    print(f"{model_name} - Root Mean Squared Error (RMSE): {rmse}, R2: {r2}")

# Stop the Spark session
spark.stop()


RandomForestRegressor - Root Mean Squared Error (RMSE): 20.4463633834094, R2: 0.734799143742128
LinearRegression - Root Mean Squared Error (RMSE): 14.094089967195988, R2: 0.8739865648412362
GBTRegressor - Root Mean Squared Error (RMSE): 15.991982332136542, R2: 0.8377639460490582
LinearRegression - Root Mean Squared Error (RMSE): 14.072926181899728, R2: 0.8743647260362339
